In [ ]:
import requests
import csv

In [ ]:
# TODO: fetch data from NYT API, save ISBN in list
# iterate over list, fetch details for each ISBN
# write to CSV

# can we get all data from open library?
    # Title, subtitle, authors, publisher, publish_data, subjects, notes(?), cover
    # from NYTimes: description

In [ ]:
ISBNs = []
books = []
descriptions = {}

In [ ]:
API_KEY= os.environ['NYT_API_KEY']
OPEN_LIBRARY_API='https://openlibrary.org/api/books?bibkeys=ISBN:{0}&format=json&jscmd=data'

NYT_API_URL='https://api.nytimes.com/svc/books/v3/lists/full-overview.json?published-date={0}&api-key={1}'



In [ ]:
YEARS = ['2022-05-31', '2021-05-31', '2020-05-31', '2019-05-31', '2018-05-31']

for year in YEARS:
    r = requests.get(NYT_API_URL.format(year, API_KEY))
    data = r.json()
    print(data)
    for l in data['results']['lists']:
        for book in l['books']:
            ISBNs.append(book['primary_isbn13'])
            descriptions[book['primary_isbn13']] = book.get("description")

In [ ]:
len(ISBNs)

In [ ]:
for isbn in ISBNs:
    try:
        r = requests.get(OPEN_LIBRARY_API.format(isbn))
        print(r.json())
        data = r.json()['ISBN:{0}'.format(isbn)]
        print(data)
        bookData = {}
        bookData['isbn'] = isbn
        bookData['title'] = data.get('title')
        bookData['subtitle'] = data.get('subtitle')
        bookData['authors'] = ";".join([x.get('name') for x in data.get('authors')])
        bookData['publisher'] = data.get('publishers')[0]['name'] #";".join([x.get('name') for x in data.get('publishers')])
        bookData['publish_date'] = data.get('publish_date')
        bookData['subjects'] = ";".join([x.get('name') for x in data.get('subjects')])
        bookData['cover'] = data.get('cover').get('large')
        books.append(bookData)
    except:
        pass

In [ ]:
len(books)

In [ ]:
# Save data to csv files

In [ ]:
with open ('subjects.csv', 'w') as csvfile:
    fieldnames = ['subject', 'isbn']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for b in books:
        for s in b['subjects'].split(';'):
            writer.writerow({'subject': s, 'isbn': b['isbn']})

In [ ]:
with open ('authors.csv', 'w') as csvfile:
    fieldnames = ['author', 'isbn']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for b in books:
        for a in b['authors'].split(';'):
            writer.writerow({'author': a, 'isbn': b['isbn']})

In [ ]:
with open('publishers.csv', 'w') as csvfile:
    fieldnames = ['publisher', 'isbn']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for b in books:
        writer.writerow({'publisher': b.get('publisher'), 'isbn': b.get('isbn')})

In [ ]:
with open('books.csv', 'w') as csvfile:
    fieldnames = ['isbn', 'title', 'subtitle', 'description','authors', 'subjects', 'publisher', 'publish_date', 'cover']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for b in books:
        b['description'] = descriptions[b.get('isbn')]
        writer.writerow(b)